In [1]:
import sys

# uncomment any library you need to install and run the cell

#!{sys.executable} -m pip install numpy
#!{sys.executable} -m pip install matplotlib
#!{sys.executable} -m pip install sklearn
#!{sys.executable} -m pip install emoji
#!{sys.executable} -m pip install spacy
#!{sys.executable} -m pip install gensim

In [2]:
import os, re, gzip, json, pickle, shutil, random, joblib, csv, random

import spacy

from sklearn.feature_extraction.text import TfidfVectorizer, CountVectorizer
from sklearn.decomposition import NMF, LatentDirichletAllocation, TruncatedSVD
from sklearn.linear_model import LogisticRegression
from sklearn.svm import LinearSVC

import numpy as np
import matplotlib.pyplot as plt

from emoji import UNICODE_EMOJI

%matplotlib inline

In [3]:
#!{sys.executable} -m spacy download en_core_web_sm

In [ ]:
#
# Test module based off dev work
#

In [4]:
sys.path.append('../app')
from config import Config
from preprocessing import Preprocessing

In [5]:
config = Config()

In [6]:
preprocessor = Preprocessing(config)

In [7]:
preprocessor.extract_type_and_text()

Extracting raw troll tweets...
Extracted 2116866 troll tweets (badtabs: 0, badlines: 1, skipped nonenglish: 829340)
Extracting raw user tweets...
0
1000000
2000000
3000000
4000000
5000000
0
1000000
2000000
3000000
bad tabs: 217985, bad lines: 223
Extracted 8783464 normal user tweets (badtabs: 217985, badlines: 223)


In [7]:
preprocessor.clean_tweets()

Saving 2116866 cleaned troll tweets to ../data/troll_tweets_clean.pkl.gz
0
1000000
2000000
3000000
4000000
5000000
6000000
7000000
8000000
Saving 8783464 cleaned user tweets to ../data/user_tweets_clean.pkl.gz


In [8]:
with gzip.open(config.troll_tweet_clean_path, 'rb') as fz:
    troll_tweets = pickle.load(fz)
    
with gzip.open(config.user_tweet_clean_path, 'rb') as fz:
    user_tweets = pickle.load(fz)

In [10]:
#truncate and store smaller set
random.shuffle(troll_tweets)
config.troll_tweet_clean_path = '../data/troll_tweets_clean_small.pkl.gz'
with gzip.open(config.troll_tweet_clean_path, 'wb') as oz:
    pickle.dump(troll_tweets[:100000], oz)
        
random.shuffle(user_tweets)
config.user_tweet_clean_path = '../data/user_tweets_clean_small.pkl.gz'
with gzip.open(config.user_tweet_clean_path, 'wb') as oz:
    pickle.dump(user_tweets[:100000], oz)

In [11]:
preprocessor = Preprocessing(config)

In [12]:
preprocessor.tokenize_tweets()

Storing troll tweets tagged so far: 0 commercial	<LINK> So, did you crush your workout? <USER> @_kaskp <USER> <USER> <USER> <USER> <USER>	<LINK> So , did you crush your workout ? <USER> @_kaskp <USER> <USER> <USER> <USER> <USER>	<LINK> so , do you crush your workout ? <USER> @_kaskp <USER> <USER> <USER> <USER> <USER>	TAG RB , VBD PRP VB PRP$ NN . TAG NNP TAG TAG TAG TAG TAG	your_workout	
Storing complete 100000 tagged troll tweets
Storing user tweets tagged so far: 0 NormalUser	Congrats to Toby Warrior Shaye G. of New York, NY, the winner of the Toby Keith Norway Flyaway Sweepstakes! - <LINK>	Congrats to Toby Warrior Shaye G. of New York , NY , the winner of the Toby Keith Norway Flyaway Sweepstakes ! - <LINK>	congrat to Toby Warrior Shaye G. of New York , NY , the winner of the Toby Keith Norway Flyaway sweepstake ! - <LINK>	NNS IN NNP NNP NNP NNP IN NNP NNP , NNP , DT NN IN DT NNP NNP NNP NNP NNS . : TAG	Toby_Warrior_Shaye_G. New_York the_winner the_Toby_Keith_Norway_Flyaway_Sweepsta

In [ ]:
#
# Dev work
#

In [18]:
data_path = 'data'
troll_tweet_path = '%s/russian-troll-tweets' % data_path
norm_tweet_path = '%s/twitter_cikm_2010' % data_path

#extracted text
troll_tweet_texts_path = '%s/troll_tweets.pkl.gz' % data_path
user_tweet_texts_path = '%s/user_tweets.pkl.gz' % data_path

spacy_nlp = spacy.load('en_core_web_sm')

In [35]:
def get_tweets(troll_tweet_path, norm_tweet_path):
    troll_tweets = get_troll_tweets(troll_tweet_path)
    
    #training_set_tweets.txt, training_set_users.txt, test_set_tweets.txt, test_set_users.txt
    #15861944	5766883977	@techiepalar Please prove to me that you are a real human by following me based on this tweet? I quit TrueTwitValidation & won't use it.	2009-11-16 09:04:01
    norm_tweets = get_norm_tweets('%s/training_set_tweets.txt' % norm_tweet_path)
    print('%d norm tweets from training file' % len(norm_tweets))
    norm_tweets.extend(get_norm_tweets('%s/test_set_tweets.txt' % norm_tweet_path))
    print('%d total norm tweets' % len(norm_tweets))
    
    return troll_tweets, norm_tweets


def get_troll_tweets(troll_tweet_path):
    troll_tweets = []
    badtab=0
    badline=0
    nonenglish=0
    for fn in os.listdir(troll_tweet_path):
        if not fn.endswith('.csv'):
            continue
        with open('%s/%s' % (troll_tweet_path, fn), 'r', encoding='utf-8', newline='\n') as csvfile:
            reader = csv.reader(csvfile, delimiter=',', quotechar='"')
            header = next(reader)
            #external_author_id, author, content, region, language, publish_date, arvested_date, following,
            #followers, updates, post_type, account_type, retweet, account_category, new_june_2018, alt_external_id,
            # tweet_id, article_url, tco1_step1, tco2_step1, tco3_step1
            for fields in reader:
                if not len(fields)==21:
                    badtab+=1
                    continue
                    
                if not fields[4].lower().strip()=='english':
                    nonenglish+=1
                    continue
                
                text = fields[2].strip()
                if not text:
                    badline+=1
                    continue
                    
                troll_type = fields[13].strip()
                troll_tweets.append('%s\t%s' % (troll_type, text))
                
        print(fn, len(troll_tweets), badtab, badline, nonenglish)
    return troll_tweets

            
def get_norm_tweets(path):
    norm_tweets = []
    badtab=0
    badline=0
    
    with open(path, 'r', encoding='utf-8') as f:
        for i,line in enumerate(f):
            if i%250000==0:
                print(i)
            if not line.count('\t')==3:
                badtab+=1
                continue
            line = line.replace('\n','').split('\t')[2].strip()
            if not line:
                badline+=1
                continue
            norm_tweets.append('NormalUser\t%s' % line)
            
    print('bad tabs: %d, bad lines: %d' % (badtab, badline))
    
    return norm_tweets

In [36]:
if not os.path.exists(troll_tweet_texts_path):
    troll_tweets, norm_tweets = get_tweets(troll_tweet_path, norm_tweet_path)
    
    with gzip.open('%s/troll_tweets.pkl.gz' % data_path, 'wb') as oz:
        pickle.dump(troll_tweets, oz)
        
    with gzip.open('%s/normal_tweets.pkl.gz' % data_path, 'wb') as oz:
        pickle.dump(norm_tweets, oz)
else:
    with gzip.open('%s/troll_tweets.pkl.gz' % data_path, 'rb') as fz:
        troll_tweets = pickle.load(fz)
        
    with gzip.open('%s/normal_tweets.pkl.gz' % data_path, 'rb') as fz:
        norm_tweets = pickle.load(fz)
        
len(troll_tweets), len(norm_tweets)

IRAhandle_tweets_1.csv 243891 0 0 53639
IRAhandle_tweets_10.csv 501002 0 0 136284
IRAhandle_tweets_11.csv 751623 0 0 192906
IRAhandle_tweets_12.csv 990973 0 0 256290
IRAhandle_tweets_13.csv 1011867 0 0 265581
IRAhandle_tweets_2.csv 1262387 0 0 319645
IRAhandle_tweets_3.csv 1505289 0 0 348110
IRAhandle_tweets_4.csv 1752083 0 0 403696
IRAhandle_tweets_5.csv 1969306 0 0 460357
IRAhandle_tweets_6.csv 2227356 0 1 507917
IRAhandle_tweets_7.csv 2469708 0 1 602085
IRAhandle_tweets_8.csv 2712666 0 1 721318
IRAhandle_tweets_9.csv 2946206 0 1 829340
0
250000
500000
750000
1000000
1250000
1500000
1750000
2000000
2250000
2500000
2750000
3000000
3250000
3500000
3750000
bad tabs: 170147, bad lines: 176
3675302 norm tweets from training file
0
250000
500000
750000
1000000
1250000
1500000
1750000
2000000
2250000
2500000
2750000
3000000
3250000
3500000
3750000
4000000
4250000
4500000
4750000
5000000
bad tabs: 47838, bad lines: 47
8783464 total norm tweets


(2946206, 8783464)

In [41]:
# preproc cleaning 
# - replace links with <LINK>, replace usertags with <USER>
# - normalize spaces, quotes, double quotes, etc
def clean_tweets(troll_tweets, norm_tweets):
    clean_troll_tweets = []
    for i,troll_tweet in enumerate(troll_tweets):
        clean_troll_tweets.append(clean_tweet(troll_tweet))
        
    clean_norm_tweets = []
    for i,norm_tweet in enumerate(norm_tweets):
        clean_norm_tweets.append(clean_tweet(norm_tweet))
    
    return clean_troll_tweets, clean_norm_tweets


def clean_tweet(tweet, repl={'‘':"'", '’':"'", '“':'"', '”':'"'}):
    #normalize spaces, quotes, double quotes, etc ‘’“”
    for f,r in repl.items():
        tweet = tweet.replace(f,r)
        
    #norm spaces
    tweet = re.sub('  +', ' ', tweet)
        
    #replace links with <LINK>, replace usertags with <USER>, leave hashtags
    tweet = re.sub('https?:[^ ]+', '<LINK>', tweet)
    
    tweet = re.sub('@[a-zA-Z][^ ]+', '<USER>', tweet)
    
    clean = []
    for char in tweet:
        if char in UNICODE_EMOJI['en']:
            clean.append('<EMOJI>')
        else:
            clean.append(char)
            
    return ''.join(clean)

In [42]:
clean_troll_tweets, clean_norm_tweets = clean_tweets(troll_tweets, norm_tweets)

In [43]:
with gzip.open('%s/troll_tweets_clean.pkl.gz' % data_path, 'wb') as oz:
    pickle.dump(clean_troll_tweets, oz)

with gzip.open('%s/normal_tweets_clean.pkl.gz' % data_path, 'wb') as oz:
    pickle.dump(clean_norm_tweets, oz)

In [40]:
for i,ctt in enumerate(clean_norm_tweets):
    if i>100:
        break
    print(ctt)

NormalUser	<USER> coo. thanks. just dropped you a line.
NormalUser	<USER> shit it ain't lettin me DM you back, what's your email?
NormalUser	<USER> hey cody, quick question...can you dm me?
NormalUser	<USER> dang. you need anything? I got some left over meds!
NormalUser	maybe i'm late in the game on this one, but this lowender vst is making my apt rumble!
NormalUser	i really hope A.I. makes the most of this second chance in philly. i'm glad he's goin home.
NormalUser	<USER> danny boy! wanna check out d-nice at the afex 1 year tonight? we could pre-game at mine and walk over.
NormalUser	...and if you have ppl that you care about, make sure to let them know. life is too short to lose friends over bullshit. peace and love.
NormalUser	...that shit weighs heavy on me. take responsibility for your life. I don't blame anyone for where I am in this world...
NormalUser	...including his last failed relationship. and while I know that none of it is grounded in reality, and I actually am worried a

In [26]:
special_tags = set(['<USER>', '<LINK>', '<EMOJI>'])

In [74]:
def tokenize_tweets(troll_tweets, user_tweets,
                    get_lemmas=False, get_pos=False, get_phrases=False, get_ents=False):
    spacy_nlp.tokenizer.add_special_case(f"<USER>", [{spacy.attrs.ORTH: f"<USER>"}])
    spacy_nlp.tokenizer.add_special_case(f"<LINK>", [{spacy.attrs.ORTH: f"<LINK>"}])
    spacy_nlp.tokenizer.add_special_case(f"<EMOJI>", [{spacy.attrs.ORTH: f"<EMOJI>"}])

    tagged_troll_tweets = []
    for i,troll_tweet in enumerate(troll_tweets):
        tweet_type, tweet_text = troll_tweet.split('\t')
        toks, lemmas, pos, phrases, ents = tokenize_text(tweet_text, get_lemmas, get_pos, get_phrases, get_ents)
        tagged_troll_tweets.append('%s\t%s\t%s\t%s\t%s\t%s\t%s' % (tweet_type, tweet_text, toks, lemmas, pos, phrases, ents))
        
        if i%100000==0:
            print(i, tagged_troll_tweets[-1])
            with gzip.open('%s/troll_tweets_tagged.pkl.gz' % data_path, 'wb') as oz:
                pickle.dump(tagged_troll_tweets, oz)
        
    tagged_user_tweets = []
    for i,user_tweet in enumerate(user_tweets):
        tweet_type, tweet_text = user_tweet.split('\t')
        toks, lemmas, pos, phrases, ents = tokenize_text(tweet_text, get_lemmas, get_pos, get_phrases, get_ents)
        tagged_user_tweets.append('%s\t%s\t%s\t%s\t%s\t%s\t%s' % (tweet_type, tweet_text, toks, lemmas, pos, phrases, ents))
        
        if i%100000==0:
            print(i, tagged_user_tweets[-1])
            with gzip.open('%s/user_tweets_tagged.pkl.gz' % data_path, 'wb') as oz:
                pickle.dump(tagged_user_tweets, oz)
    
    return tagged_troll_tweets, tagged_user_tweets


def tokenize_text(text, get_lemmas=True, get_pos=True, get_phrases=True, get_ents=True):
    toks = []
    lemmas = []
    pos = []
    phrases = []
    ents = []
    
    doc = spacy_nlp(text)
    if get_phrases:
        for chunk in doc.noun_chunks:
            if not ' ' in chunk.text:
                continue
            phrases.append(chunk.text.replace(' ', '_'))
        
    ent_type=''
    ent=[]
    for tok in doc:
        toks.append(tok.text)
        
        if tok.text in special_tags:
            if get_lemmas:
                lemmas.append(tok.text)
            if get_pos:
                pos.append('TAG')
            continue
        
        if get_ents:
            if tok.ent_iob == 3: #start
                ent_type = tok.ent_type_
                ent.append(tok.text)
            elif tok.ent_iob == 1: #continue
                ent.append(tok.text)
            else:
                if ent:
                    ents.append('%s:%s' % ('_'.join(ent), ent_type))
                    ent=[]
                    ent_type=''
                    
        if get_lemmas:
            lemmas.append(tok.lemma_)
            
        if get_pos:
            pos.append(tok.tag_)
        
    toks = ' '.join(toks)
    lemmas = ' '.join(lemmas)
    pos = ' '.join(pos)
    phrases = ' '.join(phrases)
    ents = ' '.join(ents)
    
    return toks, lemmas, pos, phrases, ents

In [7]:
with gzip.open('%s/troll_tweets_clean.pkl.gz' % data_path, 'rb') as fz:
    clean_troll_tweets = pickle.load(fz)

with gzip.open('%s/user_tweets_clean.pkl.gz' % data_path, 'rb') as fz:
    clean_user_tweets = pickle.load(fz)

In [ ]:
get_lemmas=True
get_pos=True
get_phrases=True
get_ents=True

tagged_troll_tweets, tagged_user_tweets = tokenize_tweets(clean_troll_tweets, clean_user_tweets,
                                                          get_lemmas, get_pos, get_phrases, get_ents)

0 RightTroll	"We have a sitting Democrat US Senator on trial for corruption and you've barely heard a peep from the mainstream media." ~ <USER> <LINK>	" We have a sitting Democrat US Senator on trial for corruption and you 've barely heard a peep from the mainstream media . " ~ <USER> <LINK>	" we have a sit Democrat US Senator on trial for corruption and you 've barely hear a peep from the mainstream medium . " ~ <USER> <LINK>	`` PRP VBP DT VBG NNP NNP NNP IN NN IN NN CC PRP VBP RB VBN DT NN IN DT NN NNS . '' NFP TAG TAG	a_sitting_Democrat_US_Senator a_peep the_mainstream_media	Democrat_US:GPE
100000 RightTroll	1%ers defrauded People of their President in Primary. If wise they'll give We The People something if a #12thAmendmentElection doesn't occur <LINK>	1%ers defrauded People of their President in Primary . If wise they 'll give We The People something if a # 12thAmendmentElection does n't occur <LINK>	1%er defraud People of their President in Primary . if wise they 'll give we the 

In [ ]:
with gzip.open('%s/troll_tweets_tagged.pkl.gz' % data_path, 'wb') as oz:
    pickle.dump(tagged_troll_tweets, oz)
    
with gzip.open('%s/user_tweets_tagged.pkl.gz' % data_path, 'wb') as oz:
    pickle.dump(tagged_user_tweets, oz)

In [ ]:
#
# Data Analysis
#

In [15]:
# compare histograms
# - lens of troll vs normal tweets
# - user mentions, link mentions
# - given a dict, percent of words in tweet not in the dict

In [3]:
punct = set(["'", '"', '.', ',', '~', '!', '@', '#', '$', '%', '^', '&', '*', '|',
             '(', ')', '-', '_', '+', '=', '{','}','[',']',';', ':', '<', '>', '?', '/'])

In [4]:
from nltk.corpus import words
#nltk.download('words')

In [5]:
from nltk.corpus import wordnet 
#nltk.download('wordnet')

In [6]:
vocab = set(words.words()) | set(wordnet.words())
type(vocab), len(vocab) #235892  331327

(set, 331327)

In [21]:
with gzip.open('../data/troll_tweets_tagged.pkl.gz', 'rb') as fz:
    troll_tweets = pickle.load(fz)

with gzip.open('../data/user_tweets_tagged.pkl.gz', 'rb') as fz:
    user_tweets = pickle.load(fz)
len(troll_tweets), len(user_tweets)

In [7]:
def get_features(tweets, vocab, punct):
    feats = []
    for i,tweet in enumerate(tweets):
        if i and i%100000==0:
            print(i, feats[-1])
            
        tp,txt,toks,lems,pos,phrs,ents = tweet.replace('\xa0','').split('\t')
        
        if tp=='NonEnglish':
            continue
        
        num_toks = toks.count(' ')+1
        emoji_ratio, link_ratio, user_ratio = 0,0,0
        emoji_ratio = txt.count('<EMOJI>')/num_toks
        link_ratio = txt.count('<LINK>')/num_toks
        user_ratio = txt.count('<USER>')/num_toks
        
        toks = toks.replace('< ', '<').replace(' >', '>').replace('# ','#')
        
        clean_ents = set()
        ent_types = set()
        ent_toks = set()
        for ent in ents.split(' '):
            if ent=='#:CARDINAL':
                continue
            items = ent.split(':')
            for t in items[0].split('_'):
                ent_toks.add(t)
            typ = items[-1]
            if not typ:
                continue
            if typ[0]=='#':
                continue
            if '\xa0' in typ:
                continue
            clean_ents.add(ent)
            ent_types.add(typ)
        clean_ents = list(clean_ents)
        ent_types = list(ent_types)
        ent_toks = list(ent_toks)
        
        num_ents = ents.count(' ')+1
                
        lems = lems.lower()
        lems = re.sub('<[^>]+>', '', lems)
        lems = lems.replace('# ','#')
        
        voc=[]
        novoc=[]
        for lem in lems.split(' '):
            if not lem or lem in punct or lem[0]=='#' or lem in ent_toks:
                continue
            if lem in vocab:
                voc.append(lem)
            else:
                novoc.append(lem)
                
        ratio = 0
        if voc or novoc:
            ratio = len(novoc)/(len(voc)+len(novoc))
            #print('%s\nvoc: %s\nno voc: %s\nratio: %.6f\n' % (txt, voc, novoc, ratio))
            
        tags = re.findall('#[^ ]+', txt)
            
        feats.append({
            'type':tp,
            'text':txt,
            'tokens':toks,
            'lemmas':lems,
            'pos':pos,
            'phrases':phrs,
            'entities':clean_ents,
            'ent_types':ent_types,
            'hashtags':tags,
            'oov_words':' '.join(novoc),
            'emoji_ratio':emoji_ratio, 
            'link_ratio':link_ratio, 
            'user_ratio':user_ratio,
            'oov_ratio':ratio
        })
        
    return feats

In [119]:
troll_feats = get_features(troll_tweets, vocab, punct)

100000 {'type': 'RightTroll', 'text': 'California Bakersfield #breaking #VoteTrump #MakeAmericaGreatAgain #MAGA3X <LINK>', 'tokens': 'California Bakersfield #breaking #VoteTrump #MakeAmericaGreatAgain #MAGA3X <LINK>', 'lemmas': 'california bakersfield #break #votetrump #makeamericagreatagain #maga3x ', 'pos': 'NNP NNP $ VBG $ NNP NN NNP NN FW TAG', 'phrases': 'California_Bakersfield', 'entities': ['#_breaking_#:MONEY', 'California:GPE'], 'ent_types': ['GPE', 'MONEY'], 'hashtags': ['#breaking', '#VoteTrump', '#MakeAmericaGreatAgain', '#MAGA3X'], 'oov_words': '', 'emoji_ratio': 0.0, 'link_ratio': 0.09090909090909091, 'user_ratio': 0.0, 'oov_ratio': 0.0}
200000 {'type': 'RightTroll', 'text': 'RT PoliticalShort: "Kim Jong-un, the world always thought he wasn\'t a responsible leader...He\'s acting more respon… <LINK>', 'tokens': 'RT PoliticalShort : " Kim Jong - un , the world always thought he was n\'t a responsible leader ... He \'s acting more respon … <LINK>', 'lemmas': 'rt politicalsho

1500000 {'type': 'NewsFeed', 'text': '5.2 magnitude earthquake hits Borrego Springs #local #SanDiego', 'tokens': '5.2 magnitude earthquake hits Borrego Springs #local #SanDiego', 'lemmas': '5.2 magnitude earthquake hit borrego springs #local #sandiego', 'pos': 'CD NN NN VBZ NNP NNP $ JJ $ NNP', 'phrases': '5.2_magnitude_earthquake Borrego_Springs #local_#SanDiego', 'entities': ['5.2:CARDINAL', 'Borrego_Springs:FAC'], 'ent_types': ['FAC', 'CARDINAL'], 'hashtags': ['#local', '#SanDiego'], 'oov_words': 'borrego springs', 'emoji_ratio': 0.0, 'link_ratio': 0.0, 'user_ratio': 0.0, 'oov_ratio': 0.4}
1600000 {'type': 'RightTroll', 'text': 'RT magabitenews: #BREAKING: Driver of the car in #Charlottesville was a member of the left. Read More: … <LINK>', 'tokens': 'RT magabitenews : #BREAKING : Driver of the car in #Charlottesville was a member of the left . Read More : … <LINK>', 'lemmas': 'rt magabitenew : #breaking : driver of the car in #charlottesville be a member of the left . read more : …

2900000 {'type': 'NewsFeed', 'text': 'Over 23,000 Customers Without Power Due To PECO Substation Fire <LINK> <LINK>', 'tokens': 'Over 23,000 Customers Without Power Due To PECO Substation Fire <LINK> <LINK>', 'lemmas': 'over 23,000 customer without power due to peco substation fire  ', 'pos': 'IN CD NNS IN NNP IN IN NNP NNP NNP TAG TAG', 'phrases': 'Over_23,000_Customers PECO_Substation_Fire', 'entities': [], 'ent_types': [], 'hashtags': [], 'oov_words': '23,000 peco', 'emoji_ratio': 0.0, 'link_ratio': 0.16666666666666666, 'user_ratio': 0.0, 'oov_ratio': 0.2}


In [120]:
with gzip.open('../data/troll_features.pkl.gz', 'wb') as oz:
    pickle.dump(troll_feats, oz)

In [8]:
with gzip.open('../data/user_tweets_tagged.pkl.gz', 'rb') as fz:
    user_tweets = pickle.load(fz)
len(user_tweets)

2500001

In [9]:
user_feats = get_features(user_tweets[:2500000], vocab, punct)

100000 {'type': 'NormalUser', 'text': '<USER> it was part of her "new music generator" feature. Her only knowledge was Marr being in them, probably thought he founded them', 'tokens': '<USER> it was part of her " new music generator " feature . Her only knowledge was Marr being in them , probably thought he founded them', 'lemmas': ' it be part of her " new music generator " feature . her only knowledge be marr be in they , probably think he found they', 'pos': "XX ADD XX PRP VBD NN IN PRP$ `` JJ NN NN '' NN . PRP$ JJ NN VBD NNP VBG IN PRP , RB VBD PRP VBD PRP", 'phrases': 'her_"new_music_generator"_feature Her_only_knowledge', 'entities': ['Marr:PERSON'], 'ent_types': ['PERSON'], 'hashtags': [], 'oov_words': 'marr', 'emoji_ratio': 0.0, 'link_ratio': 0.0, 'user_ratio': 0.034482758620689655, 'oov_ratio': 0.045454545454545456}
200000 {'type': 'NormalUser', 'text': '<USER> oh perdon, puedes intentar en facebook si quieres?', 'tokens': '<USER> oh perdon , puedes intentar en facebook si qui

1500000 {'type': 'NormalUser', 'text': '<USER> The good news is, BOTH names were acceptable in the broadleaf ID test I probably just flunked.', 'tokens': '<USER> The good news is , BOTH names were acceptable in the broadleaf ID test I probably just flunked .', 'lemmas': ' the good news be , both name be acceptable in the broadleaf id test i probably just flunk .', 'pos': 'XX ADD XX DT JJ NN VBZ , DT NNS VBD JJ IN DT NNP NNP NN PRP RB RB VBN .', 'phrases': 'The_good_news BOTH_names the_broadleaf_ID_test', 'entities': [], 'ent_types': [], 'hashtags': [], 'oov_words': '', 'emoji_ratio': 0.0, 'link_ratio': 0.0, 'user_ratio': 0.045454545454545456, 'oov_ratio': 0.0}
1600000 {'type': 'NormalUser', 'text': 'Monsignor Murphys on #Yelp: Friends and I were sent here for drinks while waiting for a table at the oh so popular C... <LINK>', 'tokens': 'Monsignor Murphys on #Yelp : Friends and I were sent here for drinks while waiting for a table at the oh so popular C ... <LINK>', 'lemmas': 'monsignor

In [13]:
with gzip.open('../data/user_features.pkl.gz', 'wb') as oz:
    pickle.dump(user_feats, oz)

In [ ]:
#
# test some sklearn models
#

In [3]:
with gzip.open('../data/troll_features.pkl.gz', 'rb') as fz:
    troll_feats = pickle.load(fz)
troll_feats = troll_feats[:500000]
len(troll_feats)

500000

In [4]:
with gzip.open('../data/user_features.pkl.gz', 'rb') as fz:
    user_feats = pickle.load(fz)
user_feats = user_feats[:len(troll_feats)]
len(user_feats)

500000

In [5]:
random.shuffle(troll_feats)
random.shuffle(user_feats)

In [6]:
feats = troll_feats
while user_feats:
    feats.append(user_feats.pop())
random.shuffle(feats)
len(feats)

1000000

In [7]:
y = [1 if f['type']=='NormalUser' else 0 for f in feats]

In [8]:
with gzip.open('../data/1000000_features_x.pkl.gz', 'wb') as oz:
    pickle.dump(feats, oz)

In [9]:
with gzip.open('../data/1000000_features_y.pkl', 'wb') as o:
    pickle.dump(y, o)

In [10]:
texts = [f['lemmas'].strip() for f in feats]

In [11]:
feats=troll_feats=user_feats=None

In [13]:
tfidf_vectorizer = TfidfVectorizer(max_features=50000, stop_words="english")
tfidf_vectorizer.fit(texts)

In [14]:
tfidf = tfidf_vectorizer.fit_transform(texts)

In [15]:
joblib.dump(tfidf, '1000000_tfidf_lemmas.joblib') 

['1000000_tfidf_lemmas.joblib']

In [18]:
svd = TruncatedSVD(256)
lsa = svd.fit_transform(tfidf)
sum(svd.explained_variance_ratio_)

0.20754998021218518

In [23]:
tfidf.shape, lsa.shape

((1000000, 50000), (1000000, 256))

In [19]:
joblib.dump(lsa, '1000000_lsa256_lemmas.joblib') 

['1000000_lsa256_lemmas.joblib']

In [24]:
x_lsa_train = lsa[:-10000,:]
y_lsa_train = np.array(y[:-10000])
x_lsa_test = lsa[-10000:,:]
y_lsa_test = np.array(y[-10000:])

x_lsa_train.shape

(990000, 256)

In [25]:
svm = LinearSVC()

In [26]:
svm.fit(x_lsa_train, y_lsa_train)
svm.score(x_lsa_test, y_lsa_test)

0.7964

In [27]:
x_tfidf_train = tfidf[:-10000,:]
y_tfidf_train = np.array(y[:-10000])
x_tfidf_test = tfidf[-10000:,:]
y_tfidf_test = np.array(y[-10000:])
x_tfidf_train.shape

(990000, 50000)

In [28]:
svm.fit(x_tfidf_train, y_tfidf_train)
svm.score(x_tfidf_test, y_tfidf_test)

0.8943

In [32]:
coefargs = np.argsort(svm.coef_)#.tolist()[::-1]
coefargs.shape

(1, 50000)

In [34]:
svm.coef_[0,coefargs[-10:]]

array([[-6.65339554, -6.36759972, -5.98454767, ...,  3.66902463,
         3.73444047,  4.86132776]])

In [39]:
most_important_tokens = coefargs[-100:][0].tolist()[::-1]

In [48]:
voc = tfidf_vectorizer.get_feature_names()
for i, word_idx in enumerate(most_important_tokens):
    print(i, voc[word_idx])

0 com
1 10news
2 nctime
3 www
4 h1n1
5 youmightbealiberal
6 hcr
7 cofc
8 stupak
9 awsr
10 proverbs28
11 proverbs29
12 2009
13 acorn
14 39
15 worldcupdraw
16 tbot
17 dawgfootball
18 youraudiofix
19 2010
20 menino
21 dbj
22 kt
23 clench
24 upi
25 olemiss
26 cejapa
27 bettor
28 cagop
29 8203
30 parkville
31 corzine
32 shjob
33 fthood
34 hanley
35 dawgfan
36 k4c
37 prisonindustrialcomplex
38 music4games
39 payitforwardfun
40 scozzafava
41 amdt
42 09
43 drifter51
44 odetose
45 dpost
46 worldsthinnestbook
47 foreclosure
48 ga09
49 mobsterworld
50 ksm
51 cadebate
52 noh8
53 forthood
54 iranelection
55 wwiimuseum
56 blog
57 newsvine
58 buzzflash
59 efcg
60 washooting
61 swine
62 psfk
63 rvtip
64 cnen
65 ps3
66 chs
67 ocra
68 opportuniy
69 businessonline
70 apos
71 openwebaward
72 washoote
73 johann3216
74 latism
75 thepatriotjournal
76 nasatweetup
77 horry
78 allvolunteerforce
79 indymedia
80 n9ne
81 infos
82 ny23
83 appt
84 palin
85 hasan
86 berlinwall
87 sayfie
88 docluv
89 140conf
90 lastfm

865 weis
866 wheretheydothatat
867 motocross
868 karina
869 radric
870 ogilvy
871 goe
872 flashforward
873 aspca
874 turnon
875 toolbar
876 ick
877 roo
878 ocho
879 motha
880 tokio
881 firms
882 rockingham
883 edem
884 mp3
885 txte
886 extraordinaire
887 m6
888 bif5
889 diego
890 cmj
891 o_o
892 printable
893 proprietary
894 tinychat
895 cowfilm
896 nidal
897 leno
898 gretchen
899 titans
900 corcoran
901 beck
902 umaga
903 secgen
904 strumpet
905 caitlin
906 ahmadinejad
907 intuit
908 odb
909 dysfamwk
910 arouse
911 devout
912 mezvinsky
913 jm
914 musicskins
915 weakass
916 hooyah
917 frankel
918 clunker
919 patersoninbrooklyn
920 moca
921 perspectives
922 razorfish
923 skatepark
924 telco
925 carm
926 neu
927 rocawear
928 pac10
929 whatchu
930 gc
931 xiii
932 dlc
933 fillmore
934 319
935 procrastinate
936 chosen
937 gases
938 benign
939 unh
940 catalina
941 nkh
942 chillen
943 40pm
944 appleton
945 torchwood
946 dugard
947 yogananda
948 fifty100
949 kiddin
950 hokies
951 muah
952 lmmf

1594 wok
1595 leander
1596 dungy
1597 hahaa
1598 kronos
1599 wmg
1600 bestfeele
1601 rodan
1602 thekiller
1603 pdc09
1604 vtpk
1605 affiliate
1606 ibrahimovic
1607 timeshare
1608 leotard
1609 impressionist
1610 verne
1611 brenner
1612 w00
1613 waitley
1614 mcnair
1615 techcrunch
1616 edufair
1617 xoxox
1618 greasemonkey
1619 tallahassee
1620 lyricist
1621 hatin
1622 bulletin
1623 scrobble
1624 dmca
1625 arealwife
1626 994
1627 basset
1628 refreshmiami
1629 initiatives
1630 potrero
1631 tp
1632 nbs
1633 mex
1634 shrm
1635 rachelle
1636 dyer
1637 uve
1638 floridaspl
1639 ponzi
1640 thingsilearnedfromjamesbond
1641 e2
1642 kqed
1643 steny
1644 impend
1645 hdr
1646 bostonisbetter
1647 networking
1648 evar
1649 improves
1650 trenches
1651 ohh
1652 collier
1653 rpm
1654 slr
1655 peed
1656 mutha
1657 swarovski
1658 dobbs
1659 zoe
1660 foramilliondollar
1661 vod
1662 estimates
1663 scarily
1664 sojourn
1665 provid
1666 ddb
1667 agile
1668 adweek
1669 koss
1670 ahh
1671 sidewiki
1672 mbp
1673 c

2382 convicted
2383 corvette
2384 complimentary
2385 aia
2386 digium
2387 2ba
2388 whitacre
2389 lettertomyex
2390 atlantis
2391 saveballoonboy
2392 bowden
2393 earmark
2394 betting
2395 pdf
2396 lymelife
2397 leaner
2398 stossel
2399 git
2400 wec
2401 katz
2402 bein
2403 embryonic
2404 detyme
2405 slips
2406 twoddler
2407 interactive
2408 livemocha
2409 _nemesis
2410 shih
2411 flagstaff
2412 allll
2413 dx_f
2414 slumdog
2415 opel
2416 greenbuild
2417 mlpa
2418 qik
2419 agilence
2420 sebastopol
2421 fees
2422 reuse
2423 gfoc09
2424 residual
2425 thankz
2426 ennio
2427 standin
2428 payitforward
2429 asthmatic
2430 distributed
2431 glasow
2432 lk
2433 ily
2434 cyberspace
2435 customize
2436 ghosts
2437 gophers
2438 ats
2439 ashoka
2440 calswap
2441 carbs
2442 udoka
2443 karr
2444 cal
2445 stache
2446 ronson
2447 idealab
2448 arrowhead
2449 ale
2450 monique
2451 ftw
2452 abilities
2453 comreal
2454 videos
2455 myths
2456 usin
2457 armin
2458 ths
2459 podcamp
2460 discounts
2461 xd
2462 he

3212 seas
3213 overnite
3214 cliq
3215 indo
3216 amawmi
3217 vernacular
3218 mimasummit
3219 fondo
3220 jone
3221 harem
3222 hoh
3223 playwatchvilla
3224 266
3225 rates
3226 pauls
3227 perriello
3228 ghandi
3229 api
3230 okstate
3231 liberator
3232 eeoc
3233 gwap
3234 bigchampagne
3235 cocktails
3236 pps
3237 currently
3238 strips
3239 haasdesign
3240 tumbling
3241 matisyahu
3242 mmmmm
3243 deals
3244 watering
3245 halvarez
3246 nonetheless
3247 casablancas
3248 trinket
3249 pwd
3250 lincecum
3251 penetration
3252 wc
3253 ac2
3254 kingsolver
3255 homeschool
3256 avalon
3257 carchat
3258 lello
3259 windows7
3260 edina
3261 333autismnew
3262 gregor
3263 exclusives
3264 settings
3265 cheerio
3266 ie6
3267 gamefly
3268 carbonite
3269 cct2009
3270 roadtrip
3271 sauteed
3272 carville
3273 adapter
3274 goin
3275 unfamiliar
3276 cowper
3277 louboutin
3278 12seconds
3279 deanna
3280 anthropology
3281 twitterberry
3282 yeahhhhh
3283 architects
3284 larger
3285 leighton
3286 trotter
3287 youfailb

...

46991 lynnepatton
46992 steinbaraka
46993 comermd
46994 machado
46995 deniers
46996 lent
46997 mosul
46998 hetero
46999 makeamovielessromantic
47000 blackhawks
47001 overwatch
47002 inconsistency
47003 thetrumpbureau
47004 savagery
47005 blot
47006 hahn
47007 salling
47008 pusha
47009 whytrumpcanceledrallie
47010 cheryl_kernot
47011 lennaleprena
47012 lillard
47013 tonyabbottmhr
47014 newsalert
47015 neutralize
47016 geils
47017 redress
47018 renton
47019 repcohen
47020 demagoguery
47021 howtoconfuseamillennial
47022 mateen
47023 saveaca
47024 sharknado
47025 brookhaven
47026 spon
47027 vigil
47028 parkinson
47029 obamaday
47030 ajamubaraka
47031 rower
47032 rand
47033 protector
47034 feminism
47035 fatally
47036 worldmentalhealthday
47037 grieving
47038 jbwredsox
47039 _proud_american
47040 hrtablaze
47041 faithgoldy
47042 tulsigabbard
47043 colerain
47044 foodpoisone
47045 whatihatein5words
47046 basta
47047 baltimoreriot
47048 doj
47049 truthfeednews
47050 vimpelcom
47051 winburn
47

47924 pewdiepie
47925 trumpinpicture
47926 stacylstile
47927 stooge
47928 asinine
47929 signsyoureaberniesupporter
47930 trumpster
47931 rhoa
47932 millennial
47933 iwaswinninguntil
47934 bankroll
47935 ruinawestern
47936 droit
47937 fracke
47938 tuckercarlson
47939 musingsfrombarbiesdiary
47940 vfl2013
47941 prisons
47942 daywithoutimmigrant
47943 yordano
47944 starboy
47945 apd
47946 weinstein
47947 lyles
47948 godisgreat
47949 goold
47950 hyperloop
47951 politicize
47952 addabandtoatvshow
47953 ancestry
47954 getongab
47955 emmett
47956 whitneychirps
47957 ojsimpsonparole
47958 opec
47959 gamerlifein4word
47960 badjudgmentin5word
47961 happyindependenceday
47962 skyrider4438
47963 famouscreatures
47964 debates2016
47965 jailer
47966 mangrove
47967 carti
47968 fitzhunter
47969 moab
47970 racheldolezal
47971 westboro
47972 bertha
47973 detroiters
47974 mama_byll
47975 science
47976 beholden
47977 ivanka
47978 nigel_farage
47979 samdubose
47980 derp
47981 umoja
47982 zuckerberg
47983 s

48568 fellon
48569 letitplay
48570 thelastrefuge2
48571 snowflake
48572 walterscott
48573 tremz
48574 idolssa
48575 dab
48576 hollande
48577 acapa
48578 vape
48579 kraemer
48580 frfr
48581 thexfile
48582 fgm
48583 6s
48584 lamesupergroup
48585 _babyblue13
48586 deray
48587 wishiknewhowtoquit
48588 lvsoul
48589 anisiss
48590 gdot
48591 thanksobama
48592 whyitweetin5word
48593 amike4761
48594 unm
48595 insectshowsormovie
48596 tvshow
48597 parisdennard
48598 ifs
48599 listeria
48600 flintwatercrisis
48601 goducks
48602 barksdale
48603 jarreau
48604 theperfectfood
48605 0hour1
48606 windows10
48607 angrygranny1
48608 heartedness
48609 theolderweget
48610 ambientuxr
48611 takeasongfishe
48612 andris
48613 drjillstein
48614 vaccineswork
48615 1966magazine
48616 fencesmovie
48617 wethepeople
48618 my2wordnightmare
48619 lepage
48620 nationalfreedomday
48621 oscarformuhammad
48622 mccarron
48623 medicare4all
48624 cavuto
48625 9thwonder
48626 trish_regan
48627 bakedalaska
48628 llike
48629 a3

49189 bigdata
49190 getitlive
49191 andthatshowiwentviral
49192 genres
49193 showuswhatyougot
49194 cuomo
49195 _youhadonejob
49196 hastert
49197 islamic
49198 barnaby
49199 spieth
49200 thingsiwontbelieve
49201 clinton
49202 turnbullmalcolm
49203 hipstercrime
49204 basedmonitored
49205 undergroundwgn
49206 blackart
49207 hillarysoold
49208 islam
49209 foxnews
49210 renzi
49211 tooteedup
49212 blewuplikeceelosphone
49213 buzzfeed
49214 cruz
49215 hillaryforprison2016
49216 trumper
49217 christichat
49218 w_terrence
49219 weedtv
49220 impeachtrump
49221 epochtech
49222 americanhotlips
49223 makemeabenzac
49224 thisweeksgonnarockbecause
49225 henrico
49226 dtmag
49227 indiedev
49228 tpp
49229 shotsfired
49230 dejong
49231 rtifyouare
49232 bogardthat
49233 gorka
49234 chapo
49235 muhammadali
49236 johntdolan
49237 för
49238 trafficasong
49239 waynesboro
49240 quoteoftheday
49241 cruzcrew
49242 hijab
49243 xavier
49244 thursdaythought
49245 trumptrain45pac
49246 snowden
49247 deplorables
4

49804 spicer
49805 superherowedeserve
49806 aspresidentiwill
49807 wasteamillionin3word
49808 iwouldprefertoforget
49809 mikecarlton01
49810 2a
49811 mustbebanne
49812 valentinesdayin3word
49813 nativeamerican
49814 theresamechele
49815 potus
49816 transgender
49817 theresistance
49818 dumbgeniewishe
49819 uniteblue
49820 myolympicsportwouldbe
49821 thefirst100daystodolist
49822 stonedcomicbooks
49823 addpotinamovie
49824 mentalillnessicebreaker
49825 beingblackis
49826 3rdeyeplug
49827 spinrilla
49828 hipsterschoolsupplieslist
49829 mitchellvii
49830 maketvshowsaustralian
49831 japanamovie
49832 listenlive
49833 1000network
49834 itstimetopanicwhen
49835 hurricanematthew
49836 policebrutality
49837 icantevennameone
49838 12news
49839 mychildhoodhome
49840 writeapanel
49841 environment
49842 ifihadabodydouble
49843 thingspeopleontwitterlike
49844 listen2
49845 moderndeadlysin
49846 addamovieruinamovie
49847 bgn
49848 blackhistorymonth
49849 ilove__butihate
49850 improtestingbecause
498

In [43]:
voc = list(tfidf_vectorizer.vocabulary_)

8416

In [101]:
#person, gpe, org, product
et={}
for t in troll_tweets:
    ents = t.split('\t')[-1]
    if not ents:
        continue
    for ent in ents.split(' '):
        if ent=='#:CARDINAL':
            continue
        typ = ent.split(':')[-1]
        if not typ:
            continue
        if typ[0]=='#':
            continue
        if '\xa0' in typ:
            continue
        if typ not in et:
            et[typ]=0
        et[typ]+=1
et

{'CARDINAL': 303138,
 'DATE': 207540,
 'EVENT': 12242,
 'FAC': 25103,
 'GPE': 554369,
 'LANGUAGE': 1481,
 'LAW': 5641,
 'LOC': 28401,
 'MONEY': 204241,
 'NORP': 185740,
 'ORDINAL': 28766,
 'ORG': 988345,
 'PERCENT': 19105,
 'PERSON': 901969,
 'PRODUCT': 60588,
 'QUANTITY': 7449,
 'TIME': 29112,
 'WORK_OF_ART': 85598,
 'Wher_\x85': 1}

In [76]:
tp,txt,toks,lems,pos,phrs,ents = troll_tweets[3].split('\t')
toks, lems

('JUST IN : President Trump dedicates Presidents Cup golf tournament trophy to the people of Florida , Texas and Puerto Rico . <LINK>',
 'just in : President Trump dedicate Presidents Cup golf tournament trophy to the people of Florida , Texas and Puerto Rico . <LINK>')

In [80]:
def get_in_voc_ratio(lemma, voc):
    lems = []
    lemma = re.sub('< ?emoji ?>', '', lemma)
    lemma = lemma.replace('# ','#').replace('><','> <')
    for l in lemma.split(' '):
        if not l.strip():
            continue
        if l in special_tags:
            continue
        if l in punct:
            continue
        l=l.lower()
        if l[0]=='#' or l[0]=="'":
            continue
        lems.append(l)
    return [l for l in lems if l not in voc]

In [77]:
toks, lemmas, pos, phrases, ents = tokenize_text(txt, get_lemmas=True, get_pos=True, get_phrases=True, get_ents=True)
lemmas

'just in : President Trump dedicate president Cup golf tournament trophy to the people of Florida , Texas and Puerto Rico . < LINK >'

In [81]:
for i in range(30):
    typ,txt,toks,lemma,pos,phrs,ents = troll_tweets[i].split('\t')
    print(lemma, '\n', get_in_voc_ratio(lemma, voc))

" we have a sit Democrat US Senator on trial for corruption and you 've barely hear a peep from the mainstream medium . " ~ <USER> <LINK> 
 []
Marshawn Lynch arrive to game in anti - trump shirt . judge by his sag pant the shirt should say Lynch vs. belt <LINK> 
 ['marshawn', 'vs.']
Daughter of fall Navy Sailor deliver powerful monologue on anthem protest , burn her NFL packer gear . # boycottnfl <LINK> 
 ['nfl']
just in : President Trump dedicate Presidents Cup golf tournament trophy to the people of Florida , Texas and Puerto Rico . <LINK> 
 ['presidents', 'puerto']
19,000 respect our National Anthem ! # standforouranthem 🇺 🇸 <LINK> 
 ['19,000', '🇺', '🇸']
Dan Bongino : " nobody troll liberal well than Donald Trump . " exactly ! <LINK> 
 ['bongino', 'donald']
< emoji><emoji><emoji > <LINK> 
 []
' <USER> <USER> do not matter that CNN do not report on your crime . this will not change the fact that you be go down . ' 
 ['cnn']
as much as I hate promote CNN article , here they be admit e

In [16]:
# do ngram frequency analysis

In [20]:
#emojis used:
#❤️💙💜♥️⌚️🎉💥🔥♀️💯🚨⚡️🌲🙀👹🍄🌸🌻🌺🌼🌹❄️⛄️🐝🐟🐀🦌🦅🐘🦄🗽
#🙏👊👉👍👌☝️✌👋👇👏💁🏼🤷🏼🙋🚶
#😎😱😨😞☹😳😭😮😡😢😊😂🤣😅🤔🙄😬😁😖🤕😤😜
#👀⇩🎥🎣☠️💀🎶🚂👂3️⃣⏪✅➡️🔁⬇️
#✔️✖️⭕🇺🇸⚔️💣📜🎯✨☕️⚓️�⏱📖📚✪✦🎙